In [ ]:
import gluonbook as gb
from mxnet import gluon ,init,nd
from mxnet.gluon import nn
import mxnet as mx

## Dataset

In [ ]:
batch_size=256
train_iter,test_iter=gb.load_data_fashion_mnist(batch_size,resize=96)

## Densenet(model)

In [ ]:
def conv_block(num_channals):
    blk=nn.Sequential()
    blk.add(nn.BatchNorm(),nn.Activation('relu'),nn.Conv2D(num_channals,kernel_size=3,padding=1))
    return blk

In [ ]:
class DenseBlock(nn.Block):
    def __init__(self,num_convs,num_channals):
        super(DenseBlock,self).__init__()
        self.net=nn.Sequential()
        for _ in range(num_convs):
            self.net.add(conv_block(num_channals))
        
    def forward(self,X):
        for blk in self.net:
            Y=blk(X)
            X=nd.concat(X,Y,dim=1)
        return X
    

In [ ]:
def transition_block(num_channals):
    blk=nn.Sequential()
    blk.add(nn.BatchNorm(),nn.Activation('relu'),nn.Conv2D(num_channals,kernel_size=1,strides=1),nn.AvgPool2D(pool_size=2,strides=2))
    return blk


In [ ]:
net=nn.Sequential()
net.add(nn.Conv2D(64,kernel_size=7,padding=3,strides=2),
        nn.BatchNorm(),nn.Activation('relu'),
       nn.MaxPool2D(pool_size=3,padding=1,strides=2))

In [ ]:
num_channals,growth_rate=64,32
num_blk=[4,4,4,4]
for i ,num_convs in enumerate(num_blk):
    net.add(DenseBlock(num_convs,growth_rate))
    num_channals+=num_convs*growth_rate
    if i !=len(num_blk)-1:
        net.add(transition_block(num_channals//2))
    

In [ ]:
net.add(nn.BatchNorm(),nn.Activation('relu'),nn.GlobalAvgPool2D(),nn.Dense(10))

## train & eva

In [ ]:
lr,num_epochs,ctx=0.1,1,mx.gpu()
net.initialize(ctx=ctx,init=init.Xavier())
trainer=gluon.Trainer(net.collect_params(),'sgd',{'learning_rate':lr})
gb.train_ch5(net,train_iter,test_iter,batch_size,trainer,ctx,num_epochs)